# Installing

Make an environment
https://psicode.org/psi4manual/master/build_obtaining.html#conda-binary-package


# Calculate properties of H2+ with Psi4



H2+ ref
https://en.wikipedia.org/wiki/Dihydrogen_cation

I'm using tutorials from Psi4numpy

https://github.com/psi4/psi4numpy/tree/master/Tutorials

## Notes from the "Molecule Objects in Psi4" tutorial

Charge is zero by default
How to specify the charge: {charge} {multiplicity}

H2+ has one electron so it's a singlet (multiplicity is one)
H2 has two electrons so there is a triplet and singlet.

https://psicode.org/psi4manual/master/psithonmol.html#molecule-keywords

H2 bond length 0.7414 angstroms from https://cccbdb.nist.gov/exp2x.asp?casno=1333740

In [1]:

import psi4
import numpy as np

#psi4_io.set_default_path('/private/scratch/laing')

#Charge defaults to neutral 0
# Which multiplicity for H2, 3 or 1?
h2 = psi4.geometry("""
0 1
H
H 1 0.7414
units angstrom
""")
print("h2 should have a charge 0. It currently has a charge ",h2.molecular_charge())
# Set charge to 1 for h2+ 
# Spin is 1/2 so multiplicity is 
# 2S+1=2 (doublet)
h2plus = psi4.geometry("""
1 2
H
H 1 2.0
units bohr
""")
print("h2+ should have a charge 1. It currently has a charge ",h2plus.molecular_charge())

h2 should have a charge 0. It currently has a charge  0
h2+ should have a charge 1. It currently has a charge  1


The only method I could find that computed excited states energies was CI (Configuration Interaction)
https://github.com/psi4/psi4numpy/blob/e1521f25d159f5e105570bfec136dc870d76d7b5/Tutorials/09_Configuration_Interaction/9a_cis.ipynb

Have to change from RHF to UHF because RHF is for singlets only


In [2]:
import psi4
import os

# Set scratch directory in your home folder
home_dir = os.path.expanduser('~')
scratch_dir = os.path.join(home_dir, 'psi4_scratch')

# Create scratch directory if it doesn't exist
if not os.path.exists(scratch_dir):
    try:
        os.makedirs(scratch_dir, exist_ok=True)
        print(f"Created scratch directory: {scratch_dir}")
    except Exception as e:
        print(f"Error creating scratch directory: {e}")
        raise

# Reset everything
psi4.core.clean()
psi4.core.clean_options()
psi4.core.clean_timers()  # Reset timers

# Initialize Psi4
psi4.core.be_quiet()
psi4.set_memory('2 GB')
psi4.core.IOManager.shared_object().set_default_path(scratch_dir)
psi4.set_output_file('h2_excited_states.out', False)

# Define H2 molecule
h2 = psi4.geometry("""
0 1
H
H 1 0.7414
units angstrom
""")

# Set basis set and other options
psi4.set_options({
    'basis': '6-31g',
    'reference': 'rhf',
    'roots_per_irrep': 4,
    'e_convergence': 1e-6,     
    'maxiter': 100
})

try:
    # Perform EOM-CCSD calculation
    energy, wfn = psi4.energy('eom-ccsd', return_wfn=True)
    
    # Print ground state energy
    print(f"\nGround State Energy: {energy} Hartree")
    
    # Get excitation energies
    exc_energies = wfn.variable("EOM-CCSD EXCITATION ENERGIES")
    
    if exc_energies:
        print("\nExcited State Energies (eV):")
        for i, energy in enumerate(exc_energies):
            print(f"State {i+1}: {energy * 27.211386} eV")
    
except Exception as e:
    print(f"Error occurred during calculation: {str(e)}")

finally:
    # Clean up after calculation
    psi4.core.clean()
    psi4.core.clean_timers()

Created scratch directory: /Users/rob/psi4_scratch
Error occurred during calculation: 
Fatal Error: ROOTS_PER_IRREP is wrong size. Should be number of irreps.
Error occurred in file: /Users/runner/miniforge3/conda-bld/psi4_1729125684393/work/psi4/src/psi4/cc/cceom/get_eom_params.cc on line: 95
The most recent 5 function calls were:




## Moving on to "Wavefunctions in Psi4" Note that we need excited state wavefunctions

https://github.com/psi4/psi4numpy/blob/master/Tutorials/01_Psi4NumPy-Basics/1d_wavefunction.ipynb

In their notation, spin up orbitals are alpha, spin down are beta